In [13]:
import pandas as pd

In [73]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [86]:
description, index_data = [], []
with open('aaindex1') as f: 
    for i, line in enumerate(f):
        if line[0] == 'H':
            description.append(line.split()[1])
        if line[0] == 'I':
            tmp = i
        if i == tmp+1:
            index_data.append(line)
        if i == tmp+2:
            index_data.append(line)

In [84]:
aaindex_df = pd.DataFrame(description, columns=['Description'])

In [85]:
aaindex_df

,Description
0,ANDN920101
1,ARGP820101
2,ARGP820102
3,ARGP820103
4,BEGF750101
...,...
561,KARS160118
562,KARS160119
563,KARS160120
564,KARS160121


In [88]:
len(index_data)

1132